In [1]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
import sys

from PIL import Image, ImageFile
from tqdm import tqdm

from dataset import YOLODataset
import config

from utils import (
    cells_to_bboxes,
    iou_width_height as iou,
    non_max_suppression as nms,
    plot_image,
    mean_average_precision,
    cells_to_bboxes,
    get_evaluation_bboxes,
    save_checkpoint,
    load_checkpoint,
    check_class_accuracy,
    get_loaders,
    plot_couple_examples,
    convert_YOLO_pred_to_orignal_format
)

from provided_utils import (
    voc_eval,
    xy2yx,
    bbox_iou,
    eval_detection_voc,
    calc_detection_voc_ap,
    calc_detection_voc_prec_rec,
)

from loss import YoloLoss

torch.backends.cudnn.benchmark = True

ImageFile.LOAD_TRUNCATED_IMAGES = True

notebook_dir = os.getcwd()

# root_dir = os.path.abspath(os.path.join(notebook_dir, '../..'))

sys.path.append(notebook_dir)

In [2]:
print(torch.rand(5, 3))
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
print (torch.cuda.current_device())
print (torch.version.cuda)

tensor([[0.9645, 0.1800, 0.8602],
        [0.8950, 0.8829, 0.2051],
        [0.3280, 0.1086, 0.1146],
        [0.0017, 0.1479, 0.8089],
        [0.4565, 0.4135, 0.2112]])
True
NVIDIA GeForce RTX 3070 Laptop GPU
0
12.6


### Initialize the model and load trained checkpoint

In [3]:
from model_with_weights2 import YOLOv3

from train import train_fn

model = YOLOv3(num_classes = 80).to(config.DEVICE)

# Count number of trainable parameters
total_params = sum(
p.numel() for p in model.parameters() if p.requires_grad
)
total_params

61950679

In [4]:
optimizer = optim.Adam(
    model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY
)
loss_fn = YoloLoss()
scaler = torch.amp.GradScaler(device= config.DEVICE)

In [5]:
from model_with_weights2 import CNNBlock

model.layers[15].pred[1] = CNNBlock(1024, 3*(5+1), bn_act=False, kernel_size=1)
model.layers[22].pred[1] = CNNBlock(512, 3*(5+1), bn_act=False, kernel_size=1)
model.layers[29].pred[1] = CNNBlock(256, 3*(5+1), bn_act=False, kernel_size=1)

# Correct the number of classes (used to compute the forward pass)
model.num_classes= 1

model.layers[15].num_classes = 1
model.layers[22].num_classes = 1
model.layers[29].num_classes = 1

In [6]:
if config.LOAD_MODEL:
    load_checkpoint(
        checkpoint_file= "checkpointv2.pth.tar",
          model= model, 
          optimizer= optimizer, 
          lr= config.LEARNING_RATE
    )

scaled_anchors = (
    torch.tensor(config.ANCHORS)
    * torch.tensor(config.S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
).to(config.DEVICE)


model = model.to(config.DEVICE)

=> Loading checkpoint


### Create dataset and dataloaders for train, val, and test sets

In [7]:
from dataset import YOLODataset

IMAGE_SIZE = config.IMAGE_SIZE

train_csv_path=config.DATASET + "/train.csv" 
val_csv_path=config.DATASET + "/val.csv"
test_csv_path=config.DATASET + "/test.csv"

train_dataset = YOLODataset(
    train_csv_path,
    transform=config.test_transforms,
    S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
    img_dir=config.IMG_DIR,
    label_dir=config.LABEL_DIR,
    anchors=config.ANCHORS,
    )
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=False,
)

val_dataset = YOLODataset(
    val_csv_path,
    transform=config.test_transforms,
    S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
    img_dir=config.IMG_DIR,
    label_dir=config.LABEL_DIR,
    anchors=config.ANCHORS,
    )
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=False,
)

test_dataset = YOLODataset(
    test_csv_path,
    transform=config.test_transforms,
    S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
    img_dir=config.IMG_DIR,
    label_dir=config.LABEL_DIR,
    anchors=config.ANCHORS,
    )
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    shuffle=False,
    drop_last=False,
)

### Get predictions for train set, measure mAP
- We get the predictions for train set or load the ones done before as this take some time to do from scratch
- Measure mean Average Precision using our code
- Convert the format of predictions to that provided by instructors in pred_examples.npy
- Measure mean Average Precision using instructors code
This is to make sure that conversion between formats and calculation of mAP has been done properly. This way we have some confidence that you will be able to evaluate our test predictions easily

In [8]:
check_class_accuracy(model, train_loader, threshold=config.CONF_THRESHOLD)

pred_boxes, true_boxes = get_evaluation_bboxes(
    train_loader,
    model,
    iou_threshold=config.NMS_IOU_THRESH,
    anchors=config.ANCHORS,
    threshold=config.CONF_THRESHOLD,
)
mapval = mean_average_precision(
    pred_boxes,
    true_boxes,
    iou_threshold=config.MAP_IOU_THRESH,
    box_format="midpoint",
    num_classes=config.NUM_CLASSES,
)
print(f"MAP: {mapval.item()}")

100%|██████████| 64/64 [01:02<00:00,  1.02it/s]


Class accuracy is: 100.000000%
No obj accuracy is: 99.829468%
Obj accuracy is: 49.958591%


100%|██████████| 64/64 [02:14<00:00,  2.10s/it]


MAP: 0.693514347076416


Save the predictions

In [9]:
train_preds_yolo = np.array(pred_boxes)

np.save(file= r"KITTI_YOLO//train_pred_yolo", arr= train_preds_yolo)

Load the predictions

In [8]:
train_preds_yolo = np.load (file= r"KITTI_YOLO//train_pred_yolo.npy")

Convert between formats

In [ ]:
train_preds_org = convert_YOLO_pred_to_orignal_format (preds_yolo_format= train_preds_yolo)
train_preds_org = np.array(train_preds_org)

In [18]:
train_preds_org[0]

{'classes': [0, 0],
 'scores': [0.8505004048347473, 0.7564837336540222],
 'boxes': [[np.float64(0.49827513098716736),
   np.float64(0.5092000737786293),
   np.float64(0.6226857006549835),
   np.float64(0.6887157186865807)],
  [np.float64(0.48195264488458633),
   np.float64(0.4513757452368736),
   np.float64(0.6864598020911217),
   np.float64(0.6959774270653725)]]}

In [19]:
gt[0]

{'classes': [0],
 'boxes': [array([ 90.89 , 154.12 , 142.385, 210.655])],
 'truncs': ['0.00'],
 'occls': ['0']}

In [15]:
#read prediction
pred_test = train_preds_org
pred_boxes = [torch.Tensor(x["boxes"]) for x in pred_test]  # List with n_test elements [n_boxes, 4]
pred_classes = [torch.Tensor(x["classes"]) for x in pred_test] # List with n_test elements [n_boxes]
pred_scores = [torch.Tensor(x["scores"]) for x in pred_test] # List with n_test elements [n_boxes]

#read ground truth 
gt = np.load (file= r"KITTI_YOLO//y_train_.npy", allow_pickle= True)
gt_boxes = [torch.Tensor(x["boxes"]) for x in gt[1:3]]  # List with n_test elements [n_boxes, 4]
gt_labels = [torch.Tensor(x["classes"]) for x in gt[1:3]] # List with n_test elements [n_boxes]

## Calc mAP
print("Calculating mAP")
meanAP = voc_eval(pred_boxes, pred_classes, pred_scores, gt_boxes, gt_labels)["map"]
print('Mean Average Precision of best network: %.5f' % (meanAP))

Calculating mAP


ValueError: Length of input iterables need to be same.

In [12]:
train_preds_org

[{'classes': [0, 0, 0],
  'scores': [0.7884359359741211, 0.7643001079559326, 0.7324057221412659],
  'boxes': [[np.float64(0.45972119830548763),
    np.float64(0.48452921211719513),
    np.float64(0.5120295975357294),
    np.float64(0.531337782740593)],
   [np.float64(0.4758577384054661),
    np.float64(0.337467011064291),
    np.float64(0.5112063251435757),
    np.float64(0.3829282559454441)],
   [np.float64(0.48790245689451694),
    np.float64(0.13186278194189072),
    np.float64(0.5284109767526388),
    np.float64(0.19554591923952103)]]},
 {'classes': [0, 0, 0, 0, 0, 0, 0],
  'scores': [0.8255429863929749,
   0.6847116351127625,
   0.6430433988571167,
   0.6161082983016968,
   0.6154589653015137,
   0.6150676608085632,
   0.6122535467147827],
  'boxes': [[np.float64(0.5151337664574385),
    np.float64(0.576809898018837),
    np.float64(0.5673212725669146),
    np.float64(0.6434843391180038)],
   [np.float64(0.5108767244964838),
    np.float64(0.6017156951129436),
    np.float64(0.556

In [ ]:
# array([{'boxes': array([[ 91, 154, 142, 211],
#               [ 90, 100, 142, 128]]), 'classes': array([0., 0.]), 'scores': array([1. , 0.7])},